In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [ ]:
# FETCH MARKET DATA
ticker = 'AAPL'
data = yf.download(ticker, period='1y', progress=False, multi_level_index=False)
spot_price = data.Close.iat[-1]
risk_free_rate = 0.04
dividend_yield = 0.06
vol_annual = np.std(np.log(data['Close']/data["Close"].shift(1)).dropna()) * np.sqrt(252)

print(spot_price, vol_annual)

In [ ]:
# Define option parameters
start_time = time.time()
expiry_days = 60
maturity = expiry_days / 365.0 # Convert to years
strikes = np.linspace(spot_price*0.8, spot_price*1.2, 61)
steps = 1000 # Number of steps in binomial tree

print(strikes)
end_time = time.time()
print(f"{(end_time - start_time)*1000} miliseconds")

In [ ]:
def binomial_american_call(S0, K, T, r, q, sigma, N):
    dt = T / N      # Time Setup
    u = np.exp(sigma * np.sqrt(dt)) # Up Factor
    d = 1 / u       # Down Factor
    p = (np.exp((r - q) * dt) - d) / (u - d)    # Risk-neutral probability

    # Initialize stock price tree
    stock = np.zeros((N + 1, N + 1))
    for i in range (N + 1):
        for j in range (i + 1):
            stock[j, i] = S0 * (u ** (i - j)) * (d ** j)

    # Initialize option value tree
    option = np.zeros((N + 1, N + 1))
    option[:, N] = np.maximum(0, stock[:, N] - K) # Payoff at maturity

    # Backwards induction
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            exercise = np.maximum(0, stock[j, i] - K) # Early exercise value
            hold = np.exp(-r * dt) * (p * option[j, i + 1] + (1 - p) * option [j + 1, i + 1])
            option[j,i] = np.maximum(exercise, hold) # Max of exercise or hold

    return option[0, 0]

In [ ]:
option_prices = [binomial_american_call(spot_price, K, maturity, risk_free_rate, dividend_yield, vol_annual, steps) for K in strikes]

In [ ]:
plt.style.use("dark_background")
plt.figure(figsize=(8,5))
plt.plot(strikes, option_prices, label="American Option Price")
plt.scatter(strikes, option_prices, s = 12)
plt.title(f"{ticker} American Call Option Price vs Strike")
plt.xlabel("Strike Price")
plt.ylabel("Option Price (60 days to Expiration)")
plt.grid(True, alpha = 0.24)
plt.legend()
plt.tight_layout()
plt.show()